#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

In [ ]:
DATASET_DISPLAY_NAME = "[your-dataset-display-name]
MODEL_DISPLAY_NAME = '"[your-model-display-name]
ENDPOINT_DISPLAY_NAME = '"[-endpoint-display-name]"